<a href="https://colab.research.google.com/github/jinyingtld/python/blob/main/TransformerLanguageModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! nvidia-smi

Tue May 17 03:15:25 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8    12W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

We will mount a Google drive containing the data of WikiText-2

In [2]:
from google.colab import drive
drive.mount('/content/drive')
# modify the following path accordingly
# % cd '/content/drive/MyDrive/Colab Notebooks/datasets/wikitext-2'
% cd '/content/drive/My Drive/NTU_Course/AI6103/Python Examples/wikitext-2'

% ls -lah


Mounted at /content/drive
/content/drive/My Drive/NTU_Course/AI6103/Python Examples/wikitext-2
total 13M
-rw------- 1 root root  159 May 14 15:30 README
-rw------- 1 root root 1.2M May 14 15:30 test.txt
-rw------- 1 root root  11M May 14 15:30 train.txt
-rw------- 1 root root 1.1M May 14 15:30 valid.txt


In [3]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import math, copy
from torch.autograd import Variable
import matplotlib.pyplot as plt
import torch.optim as optim
import time

In [4]:
# This is the dataset classes: Dictionary for converting between the word and 
# the dictionary index. Corpus contains all the text. 
import os
from io import open
import torch

class Dictionary(object):
    def __init__(self):
        self.word2idx = {}
        self.idx2word = []

    def add_word(self, word):
        if word not in self.word2idx:
            self.idx2word.append(word)
            self.word2idx[word] = len(self.idx2word) - 1
        return self.word2idx[word]

    def __len__(self):
        return len(self.idx2word)


class Corpus(object):
    def __init__(self, path):
        self.dictionary = Dictionary()
        self.train = self.tokenize(os.path.join(path, 'train.txt'))
        self.valid = self.tokenize(os.path.join(path, 'valid.txt'))
        self.test = self.tokenize(os.path.join(path, 'test.txt'))

    def tokenize(self, path):
        """Tokenizes a text file."""
        assert os.path.exists(path)
        # Add words to the dictionary
        with open(path, 'r', encoding="utf8") as f:
            for line in f:
                words = line.split() + ['<eos>']
                for word in words:
                    self.dictionary.add_word(word)

        # Tokenize file content
        with open(path, 'r', encoding="utf8") as f:
            idss = []
            for line in f:
                words = line.split() + ['<eos>']
                ids = []
                for word in words:
                    ids.append(self.dictionary.word2idx[word])
                idss.append(torch.tensor(ids).type(torch.int64))
            ids = torch.cat(idss)

        return ids


In [7]:

# corpus = Corpus('/content/drive/MyDrive/Colab Notebooks/datasets/wikitext-2')
corpus = Corpus('/content/drive/My Drive/NTU_Course/AI6103/Python Examples/wikitext-2')

# Find all the ngrams from the text.
def find_ngram(data, max_len):
    # number of n-grams
    num_ngram = data.size(0) // max_len
    # target and input
    target = data.narrow(0, 1, num_ngram * max_len)
    data = data.narrow(0, 0, num_ngram * max_len)

    data = data.view(-1, max_len).contiguous()
    target = target.view(-1, max_len).contiguous()
    return data, target

def shuffle(input, target):
  # this is synchronized shuffling of both input tokens and targets
  assert input.shape[0] == target.shape[0]
  p = np.random.permutation(input.shape[0])
  return input[p], target[p]



# batch_size = 20
# max_len = 30
# train_data, train_target = find_ngram(corpus.train, max_len)
# train_data = train_data.cuda()
# train_target = train_target.cuda()
# val_data = batchify(corpus.valid, batch_size)
# test_data = batchify(corpus.test, batch_size)




In [6]:
def get_batch(data, batch_size, i):
  index = batch_size * i
  return data[index:index+batch_size, :]

Dataset Statistics

In [8]:
len(corpus.dictionary.idx2word)
# vocabulary size

33278

In [9]:
def test_dataset(train_data, train_target):
  print(train_data[1, :])
  print(train_target[1, :])
  print("training data shape: ", train_data.shape)
  print("training target shape: ", train_target.shape)
  shuffle(train_data, train_target)

max_len = 30
train_data, train_target = find_ngram(corpus.train, max_len)
test_dataset(train_data, train_target)

tensor([21, 22, 23,  2,  3,  4, 24, 25, 13, 26, 27, 28, 29, 30, 31, 32, 33, 34,
        35, 36, 37, 38, 39, 17, 40, 41, 15, 42, 43, 44])
tensor([22, 23,  2,  3,  4, 24, 25, 13, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
        36, 37, 38, 39, 17, 40, 41, 15, 42, 43, 44, 45])
training data shape:  torch.Size([69620, 30])
training target shape:  torch.Size([69620, 30])


The following contains code from https://github.com/hyunwoongko/transformer

In [10]:
class LayerNorm(nn.Module):
    def __init__(self, d_model, eps=1e-12):
        super(LayerNorm, self).__init__()
        self.gamma = nn.Parameter(torch.ones(d_model))
        self.beta = nn.Parameter(torch.zeros(d_model))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        # '-1' means last dimension. 

        out = (x - mean) / (std + self.eps)
        out = self.gamma * out + self.beta
        return out

In [11]:
class ScaleDotProductAttention(nn.Module):
    """
    compute scale dot product attention
    Query : given sentence that we focused on (decoder)
    Key : every sentence to check relationship with Qeury(encoder)
    Value : every sentence same with Key (encoder)
    """

    def __init__(self):
        super(ScaleDotProductAttention, self).__init__()
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, q, k, v, mask=None, e=1e-12):
        # input is 4 dimension tensor
        # [batch_size, head, length, d_tensor]
        batch_size, head, length, d_tensor = k.size()

        # 1. dot product Query with Key^T to compute similarity
        k_t = k.transpose(2, 3)  # transpose
        score = (q @ k_t) / math.sqrt(d_tensor)  # scaled dot product. 
        # Since Python 3.5, the @ operator is matmul
        # Notice that mm() is a non-broadcastable version of matmul

        # 2. apply masking (opt)
        if mask is not None:
            score = score.masked_fill(mask == 0, -1e6) 
            # put a negative number with extremely large magnitude here

        # 3. pass them softmax to make [0, 1] range
        score = self.softmax(score)

        # 4. multiply with Value
        v = score @ v

        return v, score

In [12]:
class MultiHeadAttention(nn.Module):

    def __init__(self, d_model, n_head):
        super(MultiHeadAttention, self).__init__()
        self.n_head = n_head
        self.attention = ScaleDotProductAttention()
        self.w_q = nn.Linear(d_model, d_model)
        self.w_k = nn.Linear(d_model, d_model)
        self.w_v = nn.Linear(d_model, d_model)
        self.w_concat = nn.Linear(d_model, d_model)

    def forward(self, q, k, v, mask=None):
        # 1. dot product with weight matrices
        q, k, v = self.w_q(q), self.w_k(k), self.w_v(v)

        # 2. split tensor by number of heads
        q, k, v = self.split(q), self.split(k), self.split(v)

        # 3. do scale dot product to compute similarity
        out, attention = self.attention(q, k, v, mask=mask)

        # 4. concat and pass to linear layer
        out = self.concat(out)
        out = self.w_concat(out)

        return out

    def split(self, tensor):
        """
        split tensor by number of head
        :param tensor: [batch_size, length, d_model]
        :return: [batch_size, head, length, d_tensor]
        """
        batch_size, length, d_model = tensor.size()

        d_tensor = d_model // self.n_head
        tensor = tensor.view(batch_size, length, self.n_head, d_tensor).transpose(1, 2)
        return tensor

    def concat(self, tensor):
        """
        inverse function of self.split(tensor : torch.Tensor)
        :param tensor: [batch_size, head, length, d_tensor]
        :return: [batch_size, length, d_model]
        """
        batch_size, head, length, d_tensor = tensor.size()
        d_model = head * d_tensor

        tensor = tensor.transpose(1, 2).contiguous().view(batch_size, length, d_model)
        return tensor

In [13]:
class PositionwiseFeedForward(nn.Module):

    def __init__(self, d_model, hidden, drop_prob=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.linear1 = nn.Linear(d_model, hidden)
        self.linear2 = nn.Linear(hidden, d_model)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=drop_prob)

    def forward(self, x):
        x = self.linear1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.linear2(x)
        return x

In [14]:
class PostionalEncoding(nn.Module):
    """
    compute sinusoid encoding.
    """

    def __init__(self, d_model, max_len, device):
        """
        constructor of sinusoid encoding class
        :param d_model: dimension of model
        :param max_len: max sequence length
        :param device: hardware device setting
        """
        super(PostionalEncoding, self).__init__()

        # same size with input matrix (for adding with input matrix)
        self.encoding = torch.zeros(max_len, d_model, device=device)
        self.encoding.requires_grad = False  # we don't need to compute gradient

        pos = torch.arange(0, max_len, device=device)
        pos = pos.float().unsqueeze(dim=1)
        # 1D => 2D unsqueeze to represent word's position

        _2i = torch.arange(0, d_model, step=2, device=device).float()
        # 'i' means index of d_model (e.g. embedding size = 50, 'i' = [0,50])
        # "step=2" means 'i' multiplied with two (same with 2 * i)

        self.encoding[:, 0::2] = torch.sin(pos / (10000 ** (_2i / d_model)))
        self.encoding[:, 1::2] = torch.cos(pos / (10000 ** (_2i / d_model)))
        # compute positional encoding to consider positional information of words

    def forward(self, x):
        # self.encoding
        # [max_len = 512, d_model = 512]

        batch_size, seq_len = x.size()
        # [batch_size = 128, seq_len = 30]

        return self.encoding[:seq_len, :]
        # [seq_len = 30, d_model = 512]
        # it will add with tok_emb : [128, 30, 512]

In [15]:
class TokenEmbedding(nn.Embedding):
    """
    Token Embedding using torch.nn
    they will dense representation of word using weighted matrix
    """

    def __init__(self, vocab_size, d_model):
        """
        class for token embedding that included positional information
        :param vocab_size: size of vocabulary
        :param d_model: dimensions of model
        """
        super(TokenEmbedding, self).__init__(vocab_size, d_model, padding_idx=1)

In [16]:
class TransformerEmbedding(nn.Module):
    """
    token embedding + positional encoding (sinusoid)
    positional encoding can give positional information to network
    """

    def __init__(self, vocab_size, d_model, max_len, drop_prob, device):
        """
        class for word embedding that included positional information
        :param vocab_size: size of vocabulary
        :param d_model: dimensions of model
        """
        super(TransformerEmbedding, self).__init__()
        self.tok_emb = TokenEmbedding(vocab_size, d_model)
        self.pos_emb = PostionalEncoding(d_model, max_len, device)
        self.drop_out = nn.Dropout(p=drop_prob)

    def forward(self, x):
        tok_emb = self.tok_emb(x)
        pos_emb = self.pos_emb(x)
        return self.drop_out(tok_emb + pos_emb)

In [17]:
class EncoderLayer(nn.Module):

    def __init__(self, d_model, ffn_hidden, n_head, drop_prob):
        super(EncoderLayer, self).__init__()
        self.attention = MultiHeadAttention(d_model=d_model, n_head=n_head)
        self.norm1 = LayerNorm(d_model=d_model)
        self.dropout1 = nn.Dropout(p=drop_prob)

        self.ffn = PositionwiseFeedForward(d_model=d_model, hidden=ffn_hidden, drop_prob=drop_prob)
        self.norm2 = LayerNorm(d_model=d_model)
        self.dropout2 = nn.Dropout(p=drop_prob)

    def forward(self, x, s_mask):
        # LayerNorm + self attention + dropout + skip connection
        _x = x
        x = self.norm1(x)
        x = self.attention(q=x, k=x, v=x, mask=s_mask)       
        x = self.dropout1(x) + _x
        
        # LayerNorm + FFN + dropout + skip connection
        _x = x
        x = self.norm2(x)
        x = self.ffn(x)
        x = self.dropout2(x) + _x
        return x

In [18]:
class DecoderLayer(nn.Module):

    def __init__(self, d_model, ffn_hidden, n_head, drop_prob):
        super(DecoderLayer, self).__init__()
        self.self_attention = MultiHeadAttention(d_model=d_model, n_head=n_head)
        self.norm1 = LayerNorm(d_model=d_model) 
        # should probably replace with torch.nn.LayerNorm, which is more efficient
        self.dropout1 = nn.Dropout(p=drop_prob)

        self.enc_dec_attention = MultiHeadAttention(d_model=d_model, n_head=n_head)
        self.norm2 = LayerNorm(d_model=d_model)
        # should probably replace with torch.nn.LayerNorm, which is more efficient
        self.dropout2 = nn.Dropout(p=drop_prob)

        self.ffn = PositionwiseFeedForward(d_model=d_model, hidden=ffn_hidden, drop_prob=drop_prob)
        self.norm3 = LayerNorm(d_model=d_model)
        # should probably replace with torch.nn.LayerNorm, which is more efficient
        self.dropout3 = nn.Dropout(p=drop_prob)

    def forward(self, dec, enc, t_mask, s_mask):
        # 1. compute self attention
        x = self.norm1(dec)
        x = self.self_attention(q=x, k=x, v=x, mask=t_mask)                
        x = self.dropout1(x) + dec

        if enc is not None:
            # 3. compute encoder - decoder attention
            _x = x
            x = self.norm2(x)
            x = self.enc_dec_attention(q=x, k=enc, v=enc, mask=s_mask)                       
            x = self.dropout2(x) + _x

        # 5. LayerNorm + FFN + dropout + skip
        _x = x
        x = self.norm3(x)
        x = self.ffn(x)        
        x = self.dropout3(x) + _x
        return x

In [19]:
class Encoder(nn.Module):

    def __init__(self, enc_voc_size, max_len, d_model, ffn_hidden, n_head, n_layers, drop_prob, device):
        super().__init__()
        self.emb = TransformerEmbedding(d_model=d_model,
                                        max_len=max_len,
                                        vocab_size=enc_voc_size,
                                        drop_prob=drop_prob,
                                        device=device)

        self.layers = nn.ModuleList([EncoderLayer(d_model=d_model,
                                                  ffn_hidden=ffn_hidden,
                                                  n_head=n_head,
                                                  drop_prob=drop_prob)
                                     for _ in range(n_layers)])

    def forward(self, x, s_mask):
        x = self.emb(x)

        for layer in self.layers:
            x = layer(x, s_mask)

        return x

In [20]:
class Generator(nn.Module):
    "Standard multi-class classification. "
    def __init__(self, d_model, vocab):
        super(Generator, self).__init__()
        self.proj = nn.Linear(d_model, vocab)

    def forward(self, x):
        return F.log_softmax(self.proj(x), dim=-1)

In [21]:
class TransformerLM(nn.Module):

    def __init__(self, voc_size, max_len, d_model=32, n_head=2, ffn_hidden=32, n_layers=1, drop_prob=0.1, device="cuda"):
        super().__init__()
        # self.src_pad_idx = src_pad_idx
        # self.trg_pad_idx = trg_pad_idx
        # self.trg_sos_idx = trg_sos_idx
        self.device = device
        self.encoder = Encoder(d_model=d_model,
                               n_head=n_head,
                               max_len=max_len,
                               ffn_hidden=ffn_hidden,
                               enc_voc_size=voc_size,
                               drop_prob=drop_prob,
                               n_layers=n_layers,
                               device=device)
        self.classifier = Generator(d_model, voc_size)

        # self.decoder = Decoder(d_model=d_model,
        #                        n_head=n_head,
        #                        max_len=max_len,
        #                        ffn_hidden=ffn_hidden,
        #                        dec_voc_size=dec_voc_size,
        #                        drop_prob=drop_prob,
        #                        n_layers=n_layers,
        #                        device=device)

    def forward(self, x):
        src_mask = self.make_no_peek_mask(x, x)

        # src_trg_mask = self.make_pad_mask(trg, src)

        # trg_mask = self.make_pad_mask(trg, trg) * \
        #            self.make_no_peek_mask(trg, trg)

        output = self.encoder(x, src_mask)
        output = self.classifier(output)
        # output = self.decoder(trg, enc_src, trg_mask, src_trg_mask)
        return output

    # def make_pad_mask(self, q, k):
    #     len_q, len_k = q.size(1), k.size(1)

    #     # batch_size x 1 x 1 x len_k
    #     k = k.ne(self.src_pad_idx).unsqueeze(1).unsqueeze(2)
    #     # batch_size x 1 x len_q x len_k
    #     k = k.repeat(1, 1, len_q, 1)

    #     # batch_size x 1 x len_q x 1
    #     q = q.ne(self.src_pad_idx).unsqueeze(1).unsqueeze(3)
    #     # batch_size x 1 x len_q x len_k
    #     q = q.repeat(1, 1, 1, len_k)

    #     mask = k & q
    #     return mask

    def make_no_peek_mask(self, q, k):
        len_q, len_k = q.size(1), k.size(1)

        # len_q x len_k
        # torch.tril returns the lower triangular portion of the matrix argument
        # here the argument is an all-one matrix. 
        mask = torch.tril(torch.ones(len_q, len_k)).type(torch.BoolTensor).to(self.device)

        return mask

In [22]:
###############################################################################
# Training code
###############################################################################

def evaluate(model, val_data, val_target, config):
  # Turn on evaluation mode which disables dropout.
  model.eval()
  total_loss = 0.
  
  batch_size = config['val_batch_size']
  num_data = val_data.shape[0]
  num_batch = math.ceil(num_data / batch_size)
  voc_size = config['voc_size']
  criterion = nn.NLLLoss(reduction="sum")

  with torch.inference_mode():
    for batch in range(0, num_batch):
      last_index = min(batch_size*(batch+1), num_data)
      # The last index of data that we will take
      # This is important because we cannot throw away validation or test data.
      # We can throw away training data when convenient.

      # get the data from the validation set
      input = val_data[batch_size*batch:last_index, :]
      target = val_target[batch_size*batch:last_index, :]
      output = model(input)
      output = output.view(-1, voc_size)

      total_loss += criterion(output, target.view(-1)).item()
  return total_loss / num_data / config['max_len']



def train(model, train_data, train_target, optimizer, scheduler, criterion, config):
  # shuffle the dataset. Extremely important!
  train_data, train_target = shuffle(train_data, train_target)

  model.train()
  total_loss = 0.
  batch_size = config["batch_size"]
  num_batch = train_data.shape[0] // batch_size
  start_time = time.time() # record starting time
  for batch in range(0, num_batch):
    # this is one training iteration
    input = get_batch(train_data, batch_size, batch)
    targets = get_batch(train_target, batch_size, batch)        
    model.zero_grad()
    output = model(input).reshape(-1, voc_size)
    loss = criterion(output, targets.view(-1))
    loss.backward()
    optimizer.step()
    if (scheduler is not None):
      scheduler.step()

    total_loss += loss.item()
    log_interval = config['log_interval']
    if batch % log_interval == 0 and batch > 0:
      cur_loss = total_loss / log_interval
      elapsed = time.time() - start_time
      print('| epoch {:3d} | {:5d}/{:5d} batches | lr {:02.6f} | ms/batch {:5.2f} | '
              'loss {:5.2f} | ppl {:8.2f}'.format(
          config['epoch'], batch, num_batch, optimizer.param_groups[0]['lr'],
          elapsed * 1000 / log_interval, cur_loss, math.exp(cur_loss)))
      total_loss = 0
      start_time = time.time()


# At any point you can hit Ctrl + C to break out of training early.
try:
  config = {}
  config['log_interval'] = 100
  config['min_lr'] = 1e-8
  config['max_lr'] = 1e-3
  config['weight_decay'] = 1e-4
  config['batch_size'] = 64
  config['max_len'] = 32
  config['num_epochs'] = 30
  config['voc_size'] = len(corpus.dictionary)
  config['val_batch_size'] = 128 
  # we can use larger batches during evaluation to save time
  # this is because evaluation runs in inference 
  # mode, which consumes less memory

  criterion = nn.CrossEntropyLoss(label_smoothing=0.1)


  train_data, train_target = find_ngram(corpus.train, config['max_len'])
  train_data = train_data.cuda()
  train_target = train_target.cuda()

  val_data, val_target = find_ngram(corpus.valid, config['max_len'])
  val_data = val_data.cuda()
  val_target = val_target.cuda()

  test_data, test_target = find_ngram(corpus.test, config['max_len'])
  test_data = test_data.cuda()
  test_target = test_target.cuda()

  voc_size = len(corpus.dictionary)
  model = TransformerLM(voc_size, config['max_len'], d_model=256, n_head=16, ffn_hidden=512, n_layers=3, drop_prob=0.2).cuda()
  optimizer = optim.AdamW(model.parameters(), lr=config['max_lr'], weight_decay=config['weight_decay'])
  total_steps = train_data.shape[0] // config['batch_size'] * config['num_epochs']
  scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=total_steps, eta_min=config['min_lr'])
  

  for epoch in range(1, config['num_epochs']+1):      
    epoch_start_time = time.time()
    config['epoch'] = epoch
    train(model, train_data, train_target, optimizer, scheduler, criterion, config)

    val_loss = evaluate(model, val_data, val_target, config)
    print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.2f} | '
              'valid ppl {:8.2f}'.format(epoch, (time.time() - epoch_start_time),
                                          val_loss, math.exp(val_loss)))
    
  test_loss = evaluate(model, test_data, test_target, config)
  print('| End of training | valid loss {:5.2f} | test ppl {:8.2f}'.format(test_loss, math.exp(test_loss)))


except KeyboardInterrupt:
  print('-' * 89)
  print('Exiting from training early')

| epoch   1 |   100/ 1019 batches | lr 0.001000 | ms/batch 68.67 | loss  7.71 | ppl  2229.38
| epoch   1 |   200/ 1019 batches | lr 0.001000 | ms/batch 66.97 | loss  7.06 | ppl  1161.83
| epoch   1 |   300/ 1019 batches | lr 0.001000 | ms/batch 67.45 | loss  6.88 | ppl   975.64
| epoch   1 |   400/ 1019 batches | lr 0.001000 | ms/batch 67.33 | loss  6.77 | ppl   870.61
| epoch   1 |   500/ 1019 batches | lr 0.000999 | ms/batch 67.57 | loss  6.68 | ppl   799.24
| epoch   1 |   600/ 1019 batches | lr 0.000999 | ms/batch 67.84 | loss  6.63 | ppl   759.10
| epoch   1 |   700/ 1019 batches | lr 0.000999 | ms/batch 67.89 | loss  6.59 | ppl   725.70
| epoch   1 |   800/ 1019 batches | lr 0.000998 | ms/batch 68.21 | loss  6.53 | ppl   683.99
| epoch   1 |   900/ 1019 batches | lr 0.000998 | ms/batch 68.40 | loss  6.51 | ppl   668.84
| epoch   1 |  1000/ 1019 batches | lr 0.000997 | ms/batch 68.64 | loss  6.46 | ppl   641.75
| end of epoch   1 | time: 71.17s | valid loss  5.70 | valid ppl   298